In [1]:
import re
import json
import os
import cv2
import pycocotools._mask as _mask

In [2]:
def area(rleObjs): #annotation에 들어갈 area 
    if type(rleObjs) == list:
        return _mask.area(rleObjs).tolist()
    else:
        return _mask.area([rleObjs])[0].tolist()
    
def toBbox(rleObjs): #annotation에 들어갈 bbox
    if type(rleObjs) == list:
        return _mask.toBbox(rleObjs).tolist()
    else:
        return _mask.toBbox([rleObjs])[0].tolist()

In [3]:
#ls -l | awk '{print $9}' 
list_file = []
f = open('./list_190_200.txt', 'r') #annotation 작성할 file list 텍스트 파일
lines = f.readlines()
for i in lines:
    list_file.append(i)

In [4]:
category = [{"id":1,"name":"text","supercategory":"text"},{"id":2,"name":"0","supercategory":"text"},{"id":3,"name":"1","supercategory":"text"},{"id":4,"name":"2","supercategory":"text"},{"id":5,"name":"3","supercategory":"text"},{"id":6,"name":"4","supercategory":"text"},{"id":7,"name":"5","supercategory":"text"},{"id":8,"name":"6","supercategory":"text"},{"id":9,"name":"7","supercategory":"text"},{"id":10,"name":"8","supercategory":"text"},{"id":11,"name":"9","supercategory":"text"},{"id":12,"name":"A","supercategory":"text"},{"id":13,"name":"B","supercategory":"text"},{"id":14,"name":"C","supercategory":"text"},{"id":15,"name":"D","supercategory":"text"},{"id":16,"name":"E","supercategory":"text"},{"id":17,"name":"F","supercategory":"text"},{"id":18,"name":"G","supercategory":"text"},{"id":19,"name":"H","supercategory":"text"},{"id":20,"name":"I","supercategory":"text"},{"id":21,"name":"J","supercategory":"text"},{"id":22,"name":"K","supercategory":"text"},{"id":23,"name":"L","supercategory":"text"},{"id":24,"name":"M","supercategory":"text"},{"id":25,"name":"N","supercategory":"text"},{"id":26,"name":"O","supercategory":"text"},{"id":27,"name":"P","supercategory":"text"},{"id":28,"name":"Q","supercategory":"text"},{"id":29,"name":"R","supercategory":"text"},{"id":30,"name":"S","supercategory":"text"},{"id":31,"name":"T","supercategory":"text"},{"id":32,"name":"U","supercategory":"text"},{"id":33,"name":"V","supercategory":"text"},{"id":34,"name":"W","supercategory":"text"},{"id":35,"name":"X","supercategory":"text"},{"id":36,"name":"Y","supercategory":"text"},{"id":37,"name":"Z","supercategory":"text"},{"id":38,"name":"a","supercategory":"text"},{"id":39,"name":"b","supercategory":"text"},{"id":40,"name":"c","supercategory":"text"},{"id":41,"name":"d","supercategory":"text"},{"id":42,"name":"e","supercategory":"text"},{"id":43,"name":"f","supercategory":"text"},{"id":44,"name":"g","supercategory":"text"},{"id":45,"name":"h","supercategory":"text"},{"id":46,"name":"i","supercategory":"text"},{"id":47,"name":"j","supercategory":"text"},{"id":48,"name":"k","supercategory":"text"},{"id":49,"name":"l","supercategory":"text"},{"id":50,"name":"m","supercategory":"text"},{"id":51,"name":"n","supercategory":"text"},{"id":52,"name":"o","supercategory":"text"},{"id":53,"name":"p","supercategory":"text"},{"id":54,"name":"q","supercategory":"text"},{"id":55,"name":"r","supercategory":"text"},{"id":56,"name":"s","supercategory":"text"},{"id":57,"name":"t","supercategory":"text"},{"id":58,"name":"u","supercategory":"text"},{"id":59,"name":"v","supercategory":"text"},{"id":60,"name":"w","supercategory":"text"},{"id":61,"name":"x","supercategory":"text"},{"id":62,"name":"y","supercategory":"text"},{"id":63,"name":"z","supercategory":"text"}]
#63개의 카테고리 

In [5]:
coco = {
    "images" : [],
    "annotations":[],
    "categories" : category
}

In [6]:
for num, img_name in enumerate(list_file): #images 저장
    file_ = 'D:\\synthtext\\SynthText_part\\190~200\\' + img_name.replace('\n', '')
    img = cv2.imread(file_, cv2.IMREAD_UNCHANGED)
    coco['images'].append({
        "file_name": img_name.replace('\n', ''),
        "height": img.shape[0],
        "width": img.shape[1],
        "id": num + 1
    })

In [7]:
coco['annotations'] = []

In [8]:
annotation_id = 0
for num, img_name in enumerate(list_file):# annotations 저장
    file = 'D:\\synthtext\\SynthText_GT\\190~200\\' + img_name.replace('\n', '') + '.txt'
    img_file = 'D:\\synthtext\\SynthText_part\\190~200\\' + img_name.replace('\n', '')
    img = cv2.imread(img_file)
    img_height, img_width, _ = img.shape
    f = open(file, 'r')
    lines = f.readlines()
    for i in lines:
        i = i.replace('-','')
        if i[0] == '#' or i[0] == '\n':
            continue
        k = [j.replace('\n', '') for j in i.split(',')]
        count = 0
        for m in range(int(len(k)/9) - 1):
            for category_ in coco['categories']:
                if count == 0:
                    category_width = float(k[4]) - float(k[0])
                    category_height = float(k[5]) - float(k[1])
                    segment = [[int(float(k[j])) for j in range(0, 8)]]
                    rle = _mask.frPyObjects(segment, img_height, img_width)
                    coco['annotations'].append({
                             "area": area(rle)[0],
                             "bbox": toBbox(rle)[0],
                             "category_id": 1,
                             "id": annotation_id,
                             "image_id": num + 1,
                             "iscrowd": 0,
                             "segmentation": segment
                    })
                    annotation_id += 1
                    count += 1
                else:
                    if category_['name'] in k[9*count+8]:
                        category_width = float(k[9*count+4]) - float(k[9*count])
                        category_height = float(k[9*count+5]) - float(k[9*count+1])
                        segment = [[int(float(k[j])) for j in range(9*count, 9*count+8)]]
                        rle = _mask.frPyObjects(segment, img_height, img_width)
                        coco['annotations'].append({
                            "area": area(rle)[0],
                             "bbox": toBbox(rle)[0],
                             "category_id": category_['id'],
                             "id": annotation_id,
                             "image_id": num + 1,
                             "iscrowd": 0,
                             "segmentation": segment
                        })
                        annotation_id += 1
                        count += 1
                        break
            


In [9]:
with open('synthtext_test.json', 'w') as fp: #최종 json 저장
    json.dump(coco, fp)